# Stack Overflow Analysis using SQL
In this notebook I will work with publicly available data about the technical question and answer site [Stack Overflow](https://stackoverflow.com/). I demonstrate the use of SQL to get data to answer the following questions.

  **1) What hour of the day are most questions and answers posted?**
  
  **2) Do short or long questions get more answers?**
  
  **3) Who are potential experts on the topic of BigQuery?**
  
  **4) In January 2021, did more users post questions or answers?**
  
This notebook has four sections, with each section devoted to answering one of the four questions above. 

I decided to create this notebook after completing the Kaggle course [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql). I will emulate the basic syntax demonstrated throughout that course to access data using BigQuery from the [Google Cloud](https://cloud.google.com/python/docs/reference) library. The queries implemented below demonstrate my understanding of basic SQL, covering the commands SELECT, AS, FROM, WHERE, GROUP BY, ORDER BY, and JOIN.

We begin by fetching the `stackoverflow` dataset.

In [1]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

We also import pandas, matplotlib, and seaborn.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 1) What hour of the day are most questions and answers posted?

To answer this question, we will need to access information from both the `posts_questions` and `posts_answers` tables. For both of these tables in the `stackoverflow` dataset we want to count the number of posts grouped by the hour of the day. We will limit the query to posts created in 2021.

We begin by observing the first few lines of each of the two tables to ensure that we understand the data format and column names.

In [3]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=3).to_dataframe()

In [4]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=3).to_dataframe()

We will start simply, without a JOIN statement within the SQL string. We will extract the data from the two tables separately, join the data using pandas, and then plot the data.

The column `creation_date` appears in both tables. We will count up the posts and group them by `hour_of_day`. We will also limit the query so that it only returns entries for posts created in 2021.

In [5]:
hour_answers_query = """
                SELECT EXTRACT(HOUR FROM creation_date) as hour_of_day,
                    COUNT(1) as num_answers
                FROM `bigquery-public-data.stackoverflow.posts_answers`
                WHERE creation_date >= '2021-01-01'
                    AND creation_date < '2022-01-01'
                GROUP BY hour_of_day
                ORDER BY hour_of_day
                """

# Set up the query, with a limit on the total memory accessed
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
hour_answers_query_job = client.query(hour_answers_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
hour_answers_results = hour_answers_query_job.to_dataframe() # Your code goes here

# Preview results
print(hour_answers_results.head(3))

In [6]:
hour_questions_query = """
                SELECT EXTRACT(HOUR FROM creation_date) as hour_of_day,
                    COUNT(1) as num_questions
                FROM `bigquery-public-data.stackoverflow.posts_questions`
                WHERE creation_date >= '2021-01-01'
                    AND creation_date < '2022-01-01'
                GROUP BY hour_of_day
                ORDER BY hour_of_day
                """

# Set up the query, with a limit on the total memory accessed
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
hour_questions_query_job = client.query(hour_questions_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
hour_questions_results = hour_questions_query_job.to_dataframe() # Your code goes here

# Preview results
print(hour_questions_results.head(3))

In [7]:
hour_results = hour_questions_results.join(hour_answers_results['num_answers'], on=['hour_of_day'])
hour_results.head()

In [8]:
# Plot results
sns.set_style('whitegrid')
sns.lineplot(data=hour_results.set_index('hour_of_day'))

The time of day that mosts posts are created is in the afternoon, around 2 and 3 PM. This trend holds for both questions and answers, suggesting that the site as a whole gets more traffic during these hours, and that there aren't different times of day that people like to ask questions as opposed to answering them. 

## 2) Do short or long questions get more answers?

I suspect that if the question is short then it might be easier to interpret and less complicated. If this is the case, we may expect a greater number of posts that answer the question. To consider this question we need to combine the `posts_questions` table with the `posts_answers` table and look at the number of answers per question for both short questions and long questions. We will consider a question to be short if it is under 400 characters in length and long if it has over 2000 characters. We will again limit our search to posts in 2021.

In [9]:
short_question_query = """
                SELECT que.id AS post_id,
                    COUNT(ans.creation_date) as num_answers,
                FROM `bigquery-public-data.stackoverflow.posts_questions` as que
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS ans
                    ON que.id = ans.parent_id
                WHERE que.creation_date >= '2021-01-01'
                    AND que.creation_date < '2022-01-01'
                    AND CHAR_LENGTH(que.body)<400
                GROUP BY que.id
                ORDER BY num_answers DESC
                """

# Set up the query, with a limit on the total memory accessed
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=4*10**10)
short_question_query_job = client.query(short_question_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
short_question_results = short_question_query_job.to_dataframe() # Your code goes here

# Preview results
print(short_question_results.head())

In [10]:
long_question_query = """
                SELECT que.id AS post_id,
                    COUNT(ans.creation_date) as num_answers,
                FROM `bigquery-public-data.stackoverflow.posts_questions` as que
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS ans
                    ON que.id = ans.parent_id
                WHERE que.creation_date >= '2021-01-01'
                    AND que.creation_date < '2022-01-01'
                    AND CHAR_LENGTH(que.body)>2000
                GROUP BY que.id
                ORDER BY num_answers DESC
                """

# Set up the query, with a limit on the total memory accessed
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=4*10**10)
long_question_query_job = client.query(long_question_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
long_question_results = long_question_query_job.to_dataframe() # Your code goes here

# Preview results
print(long_question_results.head())

In [11]:
proportion_ans_num_short_q = short_question_results['num_answers'].value_counts() / len(short_question_results)

In [12]:
proportion_ans_num_long_q = long_question_results['num_answers'].value_counts() / len(long_question_results)

In [13]:
type(proportion_ans_num_short_q)

In [14]:
plt.scatter(x=proportion_ans_num_short_q.index, y=proportion_ans_num_short_q)
plt.scatter(x=proportion_ans_num_long_q.index, y=proportion_ans_num_long_q)
plt.xlabel('Number of answers')
plt.ylabel('Proportion of total questions')
plt.legend(['Short questions','Long questions'])

There appears to be a minor difference between the proportion of total questions that receive just one answer, or a few answers. Short questions may be more likely to receive a larger number of answers. We would need to implement a statistical test to see if the difference is significant.

## 3) Who are potential experts on the topic of BigQuery?

To answer this question, we want a list of users that have answered many questions. The assumption will be that if a user answers many questions related to a certain topic, then they are an expert on that topic. This question mirrors the Joining Data exercise in the previously mentioned Kaggle course Intro to SQL. However, we will add important additional filter to the query. We will only only count answer posts that are of at least a minimum length. We will do this to make sure very short posts, which presumably may not answer the question or add value, are not counted in the total number of answers given by a user. Finally, we will limit our search to answers posted in 2021.

We will access information from both the `posts_questions` and `posts_answers` tables. We will group by `user_id` defined as the `owner_user_id` column from the `posts_answers` table, and count the `number_of_answers` given by each user. We will join the `posts_questions` table on the `id` column to the `parent_id` column from `posts_answers`. Additionally, we will only select questions with `tags` containing "bigquery" and having a minimum post length of 200 characters.

In [15]:
bigquery_experts_query = """
                SELECT ans.owner_user_id AS user_id,
                    COUNT(1) AS number_of_answers
                FROM `bigquery-public-data.stackoverflow.posts_answers` AS ans
                INNER JOIN `bigquery-public-data.stackoverflow.posts_questions` AS que
                    ON ans.parent_id = que.id
                WHERE que.tags LIKE '%bigquery%'
                    AND CHAR_LENGTH(ans.body) >= 200
                    AND ans.creation_date >= '2021-01-01'
                    AND ans.creation_date < '2022-02-01'
                GROUP BY user_id
                ORDER BY number_of_answers DESC
                """

In [16]:
# Set up the query, with a limit on the total memory accessed
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=3*10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() # Your code goes here

# Preview results
print(bigquery_experts_results.head())

Users with id's 5221944, 1144035, and 13473525 clearly have the highest number of posts that fit the criteria we set for length. Perhaps we should award them each a badge for **2021 Stack Overflow Expert on BigQuery**!

## 4) In January 2021, did more users post questions or answers?

To answer this question, we want to determine which users have posted a question and/or an answer in January 2021. A secondary question we want to answer is whether there more users that post questions than answers.

We will access information from both the `posts_questions` and `posts_answers` tables. We will group by `user_id` defined as the `owner_user_id` column from the `posts_answers` table, and count the `number_of_answers` given by each user. We will join the `posts_questions` table on the `id` column to the `parent_id` column from `posts_answers`, and count the `number_of_questions` given by each user.

In [17]:
question_numbers_query = """
                SELECT owner_user_id AS user_id,
                    COUNT(1) AS number_of_questions
                FROM `bigquery-public-data.stackoverflow.posts_questions`
                WHERE creation_date >= '2021-01-01'
                    AND creation_date < '2021-02-01'
                GROUP BY owner_user_id
                ORDER BY number_of_questions DESC
                """

# Set up the query, with a limit on the total memory accessed
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
question_numbers_query_job = client.query(question_numbers_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
question_numbers_results = question_numbers_query_job.to_dataframe() # Your code goes here

# Preview results
print(question_numbers_results.head())

We see that a "person" with `user_id` NaN posted the most questions! Clearly this person does not exist, but has to do with the way the data was collected/stored. Perhaps these questions were posted by visitors to the site that did not have a username. In any case, we will ignore this eccentricity in our analysis here.

In [18]:
answer_numbers_query = """
                SELECT owner_user_id AS user_id,
                    COUNT(1) AS number_of_answers
                FROM `bigquery-public-data.stackoverflow.posts_answers`
                WHERE creation_date >= '2021-01-01'
                    AND creation_date < '2021-02-01'
                GROUP BY owner_user_id
                ORDER BY number_of_answers DESC
                """

# Set up the query, with a limit on the total memory accessed
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
answer_numbers_query_job = client.query(answer_numbers_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
answer_numbers_results = answer_numbers_query_job.to_dataframe() # Your code goes here

# Preview results
print(answer_numbers_results.head())

There were far more posts by the user that answered the most questions (1020) than by the user who asked the most questions (46). 

In [19]:
q_and_a_numbers = question_numbers_results.join(answer_numbers_results['number_of_answers'], on='user_id', how='outer')
users_q_only = sum(q_and_a_numbers['number_of_answers'].isna())
users_a_only = sum(q_and_a_numbers['number_of_questions'].isna())
users_q_and_a = sum(q_and_a_numbers['number_of_questions'].notna() & q_and_a_numbers['number_of_answers'].notna())
print("Number of users that only asked questions: {}.".format(users_q_only))
print("Number of users that only answered questions: {}.".format(users_a_only))
print("Number of users that both asked and answered questions: {}.".format(users_q_and_a))
print("Total number of unique users in this dataframe: {}".format(len(q_and_a_numbers)))

More users asked questions than answer questions. A small proportion of users both asked and answered questions. For these users, which did they do more of?

In [20]:
users_q_and_a_row_TF = q_and_a_numbers['number_of_questions'].notna() & q_and_a_numbers['number_of_answers'].notna()
q_and_a_numbers[users_q_and_a_row_TF].drop('user_id', axis=1).mean()

On average, these users posted more than twice as many answers as questions.

**This is the end of the notebook. Thanks for reading!**